In [1]:
import sys
import os
import glob
import pickle
import numpy as np
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Bidirectional, Dense, Dropout, LSTM, Activation, TimeDistributed, Embedding
from keras.callbacks import ModelCheckpoint, TensorBoard, History, Callback
from keras.utils import np_utils
import re 
import pandas as pd
import music21
import IPython
from tqdm import tqdm

from playsound import playsound
from datetime import datetime
from music21 import converter, instrument, note, chord, stream
import json

Using TensorFlow backend.


In [22]:
data_directory = "/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/Model3"
data_file = "classical.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = '/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/Model3/Test3_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [23]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i + 1]
        yield X, Y

In [24]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    
    return model

In [25]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/Model3/log.csv", index = False)

In [27]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
data

"X:1 \nT:Chaconne from Partita 2 for Solo Violin (D Major section only)\nR:Classical\nC:J.S. Bach\nN:(LOTRO) Best played on lute or harp\nZ:Transcribed by Eleiniel, Brandywine server\nM:3/4\nL:1/4\nQ:1/4=60 \nK:D\n\n[D2F3/2] G/ | [CE] [A,2F3/2] G/4 A5/16 | [B,D] [G,2E3/2] F/4 G/4 | [A,F/] E/ [C2E3/2] A/ | [DF] [D3/2F/B/] A/ G/ [D/F/] | [EC/] B,/ [A,3/2F/4] G/4 A/4 G/4 F/ [A,/E/] |\n[DB,/] A,/ [G,3/2E/] G/4 F/4 E/ [G,/D/] | [A,G/c/] [F/d/] [A,2E3/2] A/ | [DF/] E/ D/ E/ F/ +f+[D/^G/] | [CEA/] B/ [A,3/2E3/2c/] d/ B/ [A,/G/C/] | +ff+[B,Fd/] f/ [G,3/2G3/2B3/2e/] d/ c/ [G,/G/B/] |\n[A,F/A/] [D/B/] [A,3/2E/c/] [F/d/] [G/e/] [A,/E/c/] | [dD/F/] E/ [d3/2f3/2D/] E/ F/ [G/d/f/] | [eA/c/] B/ [a3/2c/] A/ B/ [c/g/] | [d/f/] [c/e/] [B/g/] [A/f/] [G/d/b/] [F/B/a/] | [E/c/g/] [D/d/f/] [A2e2d] c |\n[D/d/4]+mp+c/4B/4A/4 ^G/4B/4d/4f/4 e/ +f+[D/^G/B/e/] | [A/c/e/^C/4]+mp+D/4C/4B,/4 A,/4C/4E/4=G/4 F/ +f+[^A,/F/c/e/] | [F/d/B/4]+mp+C/4D/4B,/4 G,/4B,/4D/4F/4 E/ +f+[^G,/E/d/] | [A,/4E/4c/4]+mp+B/4A/4B/4 c/4e/4

In [28]:
training_model(data)

Number of unique characters in our whole tunes database = 79
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             40448     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_21 (LSTM)               (16, 64, 256)             525312    
_________________________________________________________________
dropout_21 (Dropout)         (16, 64, 256)             0         
_________________________________________________________________
lstm_22 (LSTM)               (16, 64, 256)             525312    
_________________________________________________________________
dropout_22 (Dro

Batch: 1, Loss: 3.231658458709717, Accuracy: 0.1396484375
Batch: 2, Loss: 3.138866901397705, Accuracy: 0.138671875
Batch: 3, Loss: 3.1237752437591553, Accuracy: 0.16015625
Batch: 4, Loss: 3.0910093784332275, Accuracy: 0.1552734375
Batch: 5, Loss: 3.0367937088012695, Accuracy: 0.17578125
Batch: 6, Loss: 3.0433738231658936, Accuracy: 0.1669921875
Batch: 7, Loss: 3.0100245475769043, Accuracy: 0.1611328125
Batch: 8, Loss: 3.0030529499053955, Accuracy: 0.1826171875
Batch: 9, Loss: 2.992349624633789, Accuracy: 0.1689453125
Batch: 10, Loss: 3.014517068862915, Accuracy: 0.171875
Batch: 11, Loss: 3.0125155448913574, Accuracy: 0.1708984375
Batch: 12, Loss: 2.9979372024536133, Accuracy: 0.1884765625
Batch: 13, Loss: 2.9453413486480713, Accuracy: 0.1787109375
Batch: 14, Loss: 2.9813432693481445, Accuracy: 0.1826171875
Batch: 15, Loss: 2.952054500579834, Accuracy: 0.1982421875
Batch: 16, Loss: 2.989229679107666, Accuracy: 0.185546875
Batch: 17, Loss: 2.9946327209472656, Accuracy: 0.18359375
Batch: 

Batch: 30, Loss: 1.996878743171692, Accuracy: 0.435546875
Batch: 31, Loss: 1.9539663791656494, Accuracy: 0.451171875
Batch: 32, Loss: 1.771515965461731, Accuracy: 0.478515625
Batch: 33, Loss: 1.8379199504852295, Accuracy: 0.4677734375
Batch: 34, Loss: 1.8574097156524658, Accuracy: 0.44921875
Batch: 35, Loss: 1.9747141599655151, Accuracy: 0.447265625
Batch: 36, Loss: 1.913233995437622, Accuracy: 0.4423828125
Batch: 37, Loss: 1.8755102157592773, Accuracy: 0.4697265625
Batch: 38, Loss: 1.8878517150878906, Accuracy: 0.4521484375
Batch: 39, Loss: 1.8302184343338013, Accuracy: 0.4697265625
Batch: 40, Loss: 1.9028561115264893, Accuracy: 0.462890625
Batch: 41, Loss: 1.7906272411346436, Accuracy: 0.4755859375
Batch: 42, Loss: 1.7743490934371948, Accuracy: 0.498046875
Batch: 43, Loss: 1.8654711246490479, Accuracy: 0.44921875
Batch: 44, Loss: 1.8998234272003174, Accuracy: 0.4453125
Batch: 45, Loss: 1.9526304006576538, Accuracy: 0.43359375
Batch: 46, Loss: 1.95521879196167, Accuracy: 0.4404296875


Batch: 58, Loss: 1.6565771102905273, Accuracy: 0.4912109375
Batch: 59, Loss: 1.6065266132354736, Accuracy: 0.5029296875
Batch: 60, Loss: 1.6649943590164185, Accuracy: 0.494140625
Batch: 61, Loss: 1.660707950592041, Accuracy: 0.494140625
Batch: 62, Loss: 1.540785312652588, Accuracy: 0.5263671875
Batch: 63, Loss: 1.368179202079773, Accuracy: 0.576171875
Batch: 64, Loss: 1.5636473894119263, Accuracy: 0.513671875
Batch: 65, Loss: 1.484649658203125, Accuracy: 0.5478515625
Batch: 66, Loss: 1.4721002578735352, Accuracy: 0.548828125
Batch: 67, Loss: 1.656160593032837, Accuracy: 0.5126953125
Batch: 68, Loss: 1.7916224002838135, Accuracy: 0.48046875
Batch: 69, Loss: 1.6078803539276123, Accuracy: 0.5185546875
Batch: 70, Loss: 1.5186634063720703, Accuracy: 0.5546875
Batch: 71, Loss: 1.3773475885391235, Accuracy: 0.57421875
Batch: 72, Loss: 1.4751631021499634, Accuracy: 0.5361328125
Batch: 73, Loss: 1.5691773891448975, Accuracy: 0.4931640625
Batch: 74, Loss: 1.451046109199524, Accuracy: 0.525390625

Batch: 86, Loss: 1.1526849269866943, Accuracy: 0.6337890625
Batch: 87, Loss: 1.161466360092163, Accuracy: 0.6328125
Batch: 88, Loss: 1.2679462432861328, Accuracy: 0.6064453125
Batch: 89, Loss: 1.2231674194335938, Accuracy: 0.6181640625
Batch: 90, Loss: 1.228661060333252, Accuracy: 0.615234375
Batch: 91, Loss: 1.29221510887146, Accuracy: 0.587890625
Batch: 92, Loss: 1.0974726676940918, Accuracy: 0.6591796875
Batch: 93, Loss: 1.0194964408874512, Accuracy: 0.6650390625
Batch: 94, Loss: 1.1529686450958252, Accuracy: 0.640625
Batch: 95, Loss: 1.1650117635726929, Accuracy: 0.6220703125
Batch: 96, Loss: 1.202941656112671, Accuracy: 0.6025390625
Batch: 97, Loss: 1.1920256614685059, Accuracy: 0.6201171875
Batch: 98, Loss: 1.251636028289795, Accuracy: 0.591796875
Batch: 99, Loss: 1.198056936264038, Accuracy: 0.611328125
Batch: 100, Loss: 1.2957346439361572, Accuracy: 0.599609375
Batch: 101, Loss: 1.2159703969955444, Accuracy: 0.6142578125
Batch: 102, Loss: 1.2173455953598022, Accuracy: 0.6376953

...

Batch: 10, Loss: 0.242435485124588, Accuracy: 0.908203125
Batch: 11, Loss: 0.21155117452144623, Accuracy: 0.927734375
Batch: 12, Loss: 0.19548249244689941, Accuracy: 0.9287109375
Batch: 13, Loss: 0.18344922363758087, Accuracy: 0.9345703125
Batch: 14, Loss: 0.20540715754032135, Accuracy: 0.9306640625
Batch: 15, Loss: 0.19707223773002625, Accuracy: 0.9306640625
Batch: 16, Loss: 0.21805718541145325, Accuracy: 0.93359375
Batch: 17, Loss: 0.21207080781459808, Accuracy: 0.9326171875
Batch: 18, Loss: 0.19063729047775269, Accuracy: 0.935546875
Batch: 19, Loss: 0.19038617610931396, Accuracy: 0.9267578125
Batch: 20, Loss: 0.2185623049736023, Accuracy: 0.9228515625
Batch: 21, Loss: 0.21124695241451263, Accuracy: 0.92578125
Batch: 22, Loss: 0.1878751814365387, Accuracy: 0.9404296875
Batch: 23, Loss: 0.20415060222148895, Accuracy: 0.923828125
Batch: 24, Loss: 0.25665026903152466, Accuracy: 0.916015625
Batch: 25, Loss: 0.2265990972518921, Accuracy: 0.9248046875
Batch: 26, Loss: 0.2098989188671112, A

Batch: 35, Loss: 0.20861482620239258, Accuracy: 0.93359375
Batch: 36, Loss: 0.21484413743019104, Accuracy: 0.9287109375
Batch: 37, Loss: 0.20130498707294464, Accuracy: 0.927734375
Batch: 38, Loss: 0.18422700464725494, Accuracy: 0.93359375
Batch: 39, Loss: 0.182630717754364, Accuracy: 0.9384765625
Batch: 40, Loss: 0.22678537666797638, Accuracy: 0.919921875
Batch: 41, Loss: 0.21051549911499023, Accuracy: 0.9267578125
Batch: 42, Loss: 0.1907273232936859, Accuracy: 0.935546875
Batch: 43, Loss: 0.19982926547527313, Accuracy: 0.9267578125
Batch: 44, Loss: 0.19764015078544617, Accuracy: 0.9296875
Batch: 45, Loss: 0.21628962457180023, Accuracy: 0.9248046875
Batch: 46, Loss: 0.21916688978672028, Accuracy: 0.9306640625
Batch: 47, Loss: 0.19497571885585785, Accuracy: 0.9326171875
Batch: 48, Loss: 0.22689944505691528, Accuracy: 0.9287109375
Batch: 49, Loss: 0.2207934558391571, Accuracy: 0.92578125
Batch: 50, Loss: 0.169953852891922, Accuracy: 0.9453125
Batch: 51, Loss: 0.2086137980222702, Accuracy

Batch: 60, Loss: 0.17704902589321136, Accuracy: 0.9423828125
Batch: 61, Loss: 0.22054672241210938, Accuracy: 0.931640625
Batch: 62, Loss: 0.20428934693336487, Accuracy: 0.9326171875
Batch: 63, Loss: 0.1932634711265564, Accuracy: 0.935546875
Batch: 64, Loss: 0.2385597825050354, Accuracy: 0.912109375
Batch: 65, Loss: 0.19380253553390503, Accuracy: 0.9365234375
Batch: 66, Loss: 0.18457120656967163, Accuracy: 0.9404296875
Batch: 67, Loss: 0.18652929365634918, Accuracy: 0.94140625
Batch: 68, Loss: 0.21081598103046417, Accuracy: 0.9296875
Batch: 69, Loss: 0.1714889407157898, Accuracy: 0.9404296875
Batch: 70, Loss: 0.1872512400150299, Accuracy: 0.9345703125
Batch: 71, Loss: 0.17850996553897858, Accuracy: 0.9453125
Batch: 72, Loss: 0.21589791774749756, Accuracy: 0.9267578125
Batch: 73, Loss: 0.17458167672157288, Accuracy: 0.9423828125
Batch: 74, Loss: 0.18084321916103363, Accuracy: 0.9365234375
Batch: 75, Loss: 0.18169158697128296, Accuracy: 0.9365234375
Batch: 76, Loss: 0.175572007894516, Acc

Batch: 86, Loss: 0.21048617362976074, Accuracy: 0.92578125
Batch: 87, Loss: 0.19240888953208923, Accuracy: 0.931640625
Batch: 88, Loss: 0.1732483208179474, Accuracy: 0.93359375
Batch: 89, Loss: 0.21475151181221008, Accuracy: 0.931640625
Batch: 90, Loss: 0.25639665126800537, Accuracy: 0.9150390625
Batch: 91, Loss: 0.23314499855041504, Accuracy: 0.9169921875
Batch: 92, Loss: 0.2127162665128708, Accuracy: 0.9248046875
Batch: 93, Loss: 0.23148460686206818, Accuracy: 0.91796875
Batch: 94, Loss: 0.20599037408828735, Accuracy: 0.931640625
Batch: 95, Loss: 0.22016136348247528, Accuracy: 0.9189453125
Batch: 96, Loss: 0.21404044330120087, Accuracy: 0.9306640625
Batch: 97, Loss: 0.1755153238773346, Accuracy: 0.943359375
Batch: 98, Loss: 0.191127210855484, Accuracy: 0.9287109375
Batch: 99, Loss: 0.17581862211227417, Accuracy: 0.931640625
Batch: 100, Loss: 0.17239823937416077, Accuracy: 0.9404296875
Batch: 101, Loss: 0.18505150079727173, Accuracy: 0.931640625
Batch: 102, Loss: 0.16786208748817444, 

Batch: 111, Loss: 0.21455270051956177, Accuracy: 0.9208984375
Batch: 112, Loss: 0.20647786557674408, Accuracy: 0.93359375
Saved Weights at epoch 90 to file Weights_90.h5


In [29]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
    #remember, that here we haven't given return_sequences = True because here we will give only one character to generate the
    #sequence. In the end, we just have to get one output which is equivalent to getting output at the last time-stamp. So, here
    #in last layer there is no need of giving return sequences = True.
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [30]:
def generate_sequence(epoch_num, initial_index, seq_length):
    with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    model.load_weights(model_weights_directory + "Weights_{}.h5".format(epoch_num))
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
    #above code is for ignoring the starting string of a generated sequence. This is because we are passing any arbitrary 
    #character to the model for generating music. Now, the model start generating sequence from that character itself which we 
    #have passed, so first few characters before "\n" contains meaningless word. Model start generating the music rhythm from
    #next line onwards. The correct sequence it start generating from next line onwards which we are considering.
    
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    seq2 = seq1[:cnt]
    #Now our data contains three newline characters after every tune. So, the model has leart that too. So, above code is used for
    #ignoring all the characters that model has generated after three new line characters. So, here we are considering only one
    #tune of music at a time and finally we are returning it..
    
    return seq2

In [34]:
ep = int(input("1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: "))
ar = int(input("\n2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: "))
ln = int(input("\n3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: "))

music = generate_sequence(ep, ar, ln)

print("\nMUSIC SEQUENCE GENERATED: \n")

print(music)



1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: 80

2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: 78

3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: 700

MUSIC SEQUENCE GENERATED: 

 X:1
T:Cuetrine Gersc
Z: by Tiamo/Skjald
L: 1/4
Q: 1/4=100
K: c
[^a7/8^f7/8^G,7/8] c/8 [c/8z/8] [f/4z/8] [f/8z/8] [^d/4z/8] [^d7/8z3/8] [^D15/8z/2] [^G15/8z3/8] [f15/8^G15/8z/2] [^c7/8z3/8] [f11/8^G15/8z/2] [^c7/8z3/8] [F15/8z/2] [^A7/8z/2] [f7/8^G7/8z3/8] [^A7/8z/2] [=F7/4z3/8] [ez/2] [^c7/4=F7/4z/2] [^A7/8z3/8] [=F,7/4z/2] [^G11/8z3/8] [^c/2F15/8]
=c3/8 z/8 [^A11/8^F,15/8z/2] c/2 [^f7/4^D7/4z3/8] ^d/2 [=c3/4^G,7/4z3/8] [^d7/8z/2] [^C15/8z/2] [f7/8z/2] [^g7/4^c7/8z3/8] [fz/2] [^A,zz/2] [=c7/8z/2] [^f7/4^G7/4z3/8] [cz/2] [^D7/4z3/8] [c7/8z/2]
[=f11/8^G,7/